In [ ]:
from __future__ import annotations

import io
from contextlib import redirect_stdout

import matplotlib.patches as mpatches
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from matplotlib import cm
from mpas_tools.mesh.conversion import convert, cull
from mpas_tools.planar_hex import make_planar_hex_mesh
from mpas_tools.translate import center
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import mosaic
from mosaic.contour import MPASContourGenerator

plt.rcParams["text.usetex"] = True

SIZE = 3.5


def gaussian(x, y, sigma=0.15, A=2.0):
    """ """
    return A * np.exp(-1.0 * (x**2 / (2 * sigma**2) + y**2 / (2 * sigma**2)))


def sinusoid(x, y, A=2.0, N=1.5):
    """ """
    x_min = np.min(x)
    y_min = np.min(y)
    x_max = np.max(x)
    y_max = np.max(y)

    L_x = (x_max - x_min) / 2
    L_y = (y_max - y_min) / 2
    return A * np.sin(N * np.pi * (x / L_x)) * np.sin(N * np.pi * (y / L_y))


def planar_hex_mesh(N=100):
    """Wrapper for MPAS-Tools functions"""
    output_capture = io.StringIO()

    # Redirect stdout to the buffer within the 'with' block
    with redirect_stdout(output_capture):
        ds = make_planar_hex_mesh(
            nx=N, ny=N, dc=1 / N, nonperiodic_x=True, nonperiodic_y=True
        )
        ds = cull(ds)
        ds = convert(ds)
        # returns None, does centering inplace
        center(ds)

    return ds


def single_pannel(figsize=(SIZE, SIZE)):
    fig, ax = plt.subplots(figsize=figsize, layout="constrained")
    ax.set_aspect("equal")
    ax.set_axis_off()
    return fig, ax


def double_pannel(figsize=(2 * SIZE, SIZE)):
    w, h = figsize
    figsize = (w + 0.1 / (2 / w), h)

    fig, axes = plt.subplots(
        nrows=1,
        ncols=3,
        figsize=(figsize),
        layout="constrained",
        width_ratios=[1, 0.1, 1],
    )

    axes[1].plot([0 for i in range(10)], range(10), c="k", lw=0.5)

    for ax in axes:
        ax.set_axis_off()
        ax.set_aspect("equal")

    ax = axes[[0, 2]]
    ax[0].sharex(ax[1])
    ax[0].sharey(ax[1])
    return fig, ax

In [ ]:
# see: https://github.com/jupyterlab/jupyterlab/issues/9309
%config InlineBackend.figure_format = 'retina'

# Contouring

**Developer**: Andrew Nolan (github:[`andrewdnolan`](https://github.com/andrewdnolan/))

## Overview
For complete visualization capabilities in `mosaic`, we need to be able to create contour and filled contour plots. Unfortunately, we are unable to use the existing [`matplotlib.pyplot.contour`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html) and [`matplotlib.pyplot.contourf`](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contourf.html) functions, due to two `matplotlib` requirements: 
  1. That the `X` and `Y` coordinates arrays represent a regularly spaces, structured grid.
  2. That the `Z` data being contoured be coincidental with the `X` and `Y` coordinates on the mesh.

The above requirements are enforced, because "under the hood" matplotlib uses the [`contourpy`](https://contourpy.readthedocs.io/) packages, which is a C++ implementation of the [marching squares](https://en.wikipedia.org/wiki/Marching_squares) algorithm. The unstrucuted nature of MPAS meshes prevents (1) from ever being satisfied. Because MPAS meshes represent control volumes, data are not defined at discrete "nodes" but over a control volumes, therefore preventing (2) from being easily satisfied. The analogous [marching triangles](https://en.wikipedia.org/wiki/Marching_squares#Contouring_triangle_meshes) algorithm could work for out unstructured data, but [`contourpy`](https://contourpy.readthedocs.io/) has not implemented it. From a cursory look, there do not seem to be any implementations of marching triangles, with a python interface, as mature and well mainted as the marching squares algorithm within [`contourpy`](https://contourpy.readthedocs.io/). 

Instead, we've opted to follow Andrew Robert's [`Ridgepack` MATLAB package](https://github.com/proteanplanet/Ridgepack) and use the mesh connectivity information to create contours of unstrucuted MPAS meshes. The implementation below follows `Ridgepack`, but we have opted to avoid a direct port of the MATLAB code. Instead we crib from the approach, but try to implement as much as possible using external python libraries to increase robustness and performance. 

## Rough Sketch of Implementation

Let's start by considering a small ($20 \times 20$ cell) planar non-periodic MPAS mesh. The initial field we will consider will be a 2D gaussian with an amplitude ($A$) of $2$ and standard deviation ($\sigma$) of $0.15$:

In [ ]:
ds = planar_hex_mesh(20)
gauss = gaussian(ds.xCell, ds.yCell)
descriptor = mosaic.Descriptor(ds)

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(ax, descriptor, gauss, vmax=2, vmin=0, aa=True)

fig.colorbar(pc, shrink=0.5, extend="both");

**Figure 1.** Gaussian kernel with $A=2$ and $\sigma=0.15$ evaluated on a $20\times20$ cell planar non-periodic MPAS mesh. 

Now, let's contour the `1.0` level. Like "Marching Squares" algorithm, we'll start by creating a boolean mask of cells above and below the contour level. 

In [ ]:
mask = gauss < 1.0

In [ ]:
edge_mask = np.logical_xor.reduce(
    mask[descriptor.ds.cellsOnEdge].values, axis=1
)

cv_edges = descriptor.ds.nEdges[edge_mask]
cv_vertices = descriptor.ds.verticesOnEdge[edge_mask]

nVertices = cv_vertices.sizes["nEdges"]

xVertex = ds.xVertex.values
yVertex = ds.yVertex.values

xv_unsorted = np.insert(xVertex[cv_vertices], 2, np.nan, axis=1)
yv_unsorted = np.insert(yVertex[cv_vertices], 2, np.nan, axis=1)

vertex_1 = cv_vertices.isel(TWO=0).values
vertex_2 = cv_vertices.isel(TWO=1).values

graph = nx.Graph()
graph.add_edges_from(zip(vertex_1, vertex_2, strict=False))
cv_euler = np.array(list(nx.eulerian_circuit(graph)))

xv_sorted = np.insert(xVertex[cv_euler], 2, np.nan, axis=1)
yv_sorted = np.insert(yVertex[cv_euler], 2, np.nan, axis=1)

lw = 2.5
cmap = cm.plasma
norm = cm.colors.Normalize(vmin=0, vmax=nVertices)
s_map = cm.ScalarMappable(norm=norm, cmap=cmap)

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(
    ax, descriptor, mask, cmap="binary_r", alpha=0.5, ec="k", lw=1
)

_xy = descriptor.cell_patches[0]
elements = [
    mpatches.Polygon(_xy, fc="white", ec="k", label="False"),
    mpatches.Polygon(_xy, fc="grey", ec="k", label="True"),
]

ax.legend(handles=elements, loc="lower left", fontsize="large", framealpha=1.0)

ax.set_xlim(-0.25, 0.25)
ax.set_ylim(-0.25, 0.25);

**Figure 2.** Boolean mask of cells with a value greater than $1$ for the field plotted above (**Figure 1**). Gray cells are above (True) and white cells are below (False) the contour level. 

From examining the boolean mask, plotted on the native mesh, it becomes clear that one way to display contour boundary is to use the edges of the mesh where the mask is true on one side and false on the other.
To identify the edges where this is true, we'll use the "exculusive or" (i.e. $\mathrm{XOR}$) operation along the second dimension of the $\mathrm{cellsOnEdge}$ array, which is of size $\mathrm{nEdges} \times 2$. 

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(
    ax, descriptor, gauss <= 1.0, cmap="binary_r", alpha=0.5, ec="k", lw=1
)

ax.plot(
    xv_unsorted.ravel(), yv_unsorted.ravel(), c="tab:blue", lw=lw, label="Edge"
)

_xy = descriptor.cell_patches[0]
elements = [
    mpatches.Polygon(_xy, fc="white", ec="k", label="True"),
    mpatches.Polygon(_xy, fc="grey", ec="k", label="False"),
]

legend = ax.legend(loc="lower right", fontsize="large", framealpha=1.0)

ax.add_artist(legend)

ax.legend(handles=elements, loc="lower left", fontsize="large", framealpha=1.0)
ax.set_xlim(-0.25, 0.25)
ax.set_ylim(-0.25, 0.25);

**Figure 3.** Boolean mask of cells with a value greater than $1$, with edges corresponding to the contour boundary denoted in blue. 

Unfortunately it's not quite that simple. While we've identified the edges that correspond to the contour boundary, the **order** of those edges are quite important for plotting.

In [ ]:
fig, ax = double_pannel()

for i in range(2):
    pc = mosaic.polypcolor(
        ax[i],
        descriptor,
        gauss <= 1.0,
        cmap="binary_r",
        alpha=0.5,
        ec="k",
        lw=1,
    )

    ax[i].set_xlim(-0.25, 0.25)
    ax[i].set_ylim(-0.25, 0.25)

for i in range(nVertices):
    c = cmap(norm(i))
    ax[0].plot(
        xVertex[cv_vertices[i]].ravel(),
        yVertex[cv_vertices[i]].ravel(),
        c=c,
        lw=lw,
    )

ax[1].plot(
    xVertex[cv_vertices].ravel(),
    yVertex[cv_vertices].ravel(),
    c="tab:blue",
    lw=lw,
    label="Unsorted",
)
legend = ax[1].legend()

cmap = cm.plasma
norm = cm.colors.Normalize(vmin=0, vmax=nVertices)
s_map = cm.ScalarMappable(norm=norm, cmap=cmap)

cax = inset_axes(
    ax[0],
    width="50%",
    height="5%",
    loc="lower left",
    bbox_to_anchor=(0.25, 0.0, 1, 1),
    bbox_transform=ax[0].transAxes,
    borderpad=0,
)

cbar = fig.colorbar(
    s_map,
    cax=cax,
    label="Edge Number",
    spacing="proportional",
    ticks=range(0, nVertices + 1, 10),
    boundaries=np.linspace(-0.5, 0.5 + nVertices, nVertices + 1),
    drawedges=True,
    ax=ax[0],
    shrink=0.5,
    orientation="horizontal",
)

**Figure 4.** (Left) Contour boundary edges color coded by their index in the boundary edge list. 
(Right) An attempt to plot the boundary edges as polygon. 
Because of the near random order of edges around the contour, the resulting polygon is self-intersecting. 

### Unstrucuted Mesh Contours as Graphs

Using the $\mathrm{XOR}$ operator and the $\mathrm{cellsOnEdge}$ connectivity array, we have identified the edges corresponding to the contour boundary.
These contour edges are useful, but for plotting purposed what we are really interested beginning and end points of these edges so that we can plot them as a line. 
We can use the $\mathrm{verticesOnEdge}$ connectivity array ($\mathrm{nEdges} \times 2$), for the subset of edges that corresponds to the contour boundary, to get the information we need.
Now this subset of $\mathrm{verticesOnEdge}$ array can be treated as a graph, where the vertices and edges of the unstrucuted mesh become the vertrices and edges of graph. 

The advantage of treating the contour boundary as a graph, is that we can leverage graph theory algorithms as implemented in [`networkx`](https://networkx.org/documentation/stable/) to efficient traverse and manipulate the contour graphs. 

**Eulerian Circuit**:  
In graph theory a [Eulerian Circuit](https://en.wikipedia.org/wiki/Eulerian_path) is path through a finite graph that visits every edge exactly once, that starts and ends on the same vertex. 
The Eulerian Circuit effectively "sorts" the graph, which we demonstrated that we need above.
The algrothimic implementation of creating a Eulerian Circuit effectively is linear in time, so it is very efficient.
Let's examine the Eulerian Circuit of the contour boundary graph we plotted above.

In [ ]:
fig, ax = double_pannel()

for i in range(2):
    pc = mosaic.polypcolor(
        ax[i],
        descriptor,
        gauss <= 1.0,
        cmap="binary_r",
        alpha=0.5,
        ec="k",
        lw=1,
    )

    ax[i].set_xlim(-0.25, 0.25)
    ax[i].set_ylim(-0.25, 0.25)

for i in range(nVertices):
    c = cmap(norm(i))

    ax[0].plot(
        xVertex[cv_euler[i]].ravel(),
        yVertex[cv_euler[i]].ravel(),
        c=c,
        lw=lw,
        label="Edge",
    )

ax[1].plot(
    xVertex[cv_euler].ravel(),
    yVertex[cv_euler].ravel(),
    c="tab:blue",
    lw=lw,
    label="Euler Circuit (ie. sorted)",
)
legend = ax[1].legend(framealpha=1.0)

cax = inset_axes(
    ax[0],
    width="50%",
    height="5%",
    loc="lower left",
    bbox_to_anchor=(0.25, 0.0, 1, 1),
    bbox_transform=ax[0].transAxes,
    borderpad=0,
)

cbar = fig.colorbar(
    s_map,
    cax=cax,
    label="Edge Number",
    spacing="proportional",
    ticks=range(0, nVertices + 1, 10),
    boundaries=np.linspace(-0.5, 0.5 + nVertices, nVertices + 1),
    drawedges=True,
    ax=ax[0],
    shrink=0.5,
    orientation="horizontal",
)

**Figure 5.** (Left) Euler circuit of contour boundary edges colorcoded by their index in the boundary edge list. (Right) Euler circuit of contour boundary edges as polygon. Because the Euler circuit is sorted, the resulting polygon is valid (i.e. not-self intersecting). 

With the Eulerian Circuit, we can now plot the contour boundary as polygon. This is the basic functionality we need to plot unstructured contour boundaries in `matplotlib`. 
Before we move on, let's quickly look at another way to deliniate the contour boundary. 

### Edge vs. Cell Contouring

Let's return to the initial boolean mask, but now with a scatter plot added discretely showing the mask values at cell centers. 

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(
    ax, descriptor, mask, cmap="binary_r", alpha=0.5, ec="k", lw=1
)
scatter = ax.scatter(
    ds.xCell, ds.yCell, c=mask, cmap="binary_r", s=2.5, ec="k", lw=0.1
)

kw = {"fmt": "{x!s:}", "func": lambda x: ~x.astype(bool)}

# produce a legend with the unique colors from the scatter
legend1 = ax.legend(
    *scatter.legend_elements(**kw), loc="lower left", title="Classes"
)
ax.add_artist(legend1)

ax.set_xlim(-0.25, 0.25)
ax.set_ylim(-0.25, 0.25);

**Figure 6.** Boolean mask of cells with a value greater than $1$, with mask values also discretly plotted at cell centers.

After some close examination, one might notice *another* way to draw contour boundary: connecting the cell centers of boundary cells where the condition is true. We will refer to this type of contour boundary as a "cell contour". Unfortunately, the connectivity logic here is not quite a simple as "edge contour". 

In [ ]:
(cell_idxs,) = np.where(~mask)

# initial edge mask, will also return all edge within the contour
# not just those along the perimiter
edge_mask = np.logical_and.reduce(
    np.isin(descriptor.ds.cellsOnEdge, cell_idxs), axis=1
)
# initial edges
cc_edges = descriptor.ds.nEdges[edge_mask]

c_edge_union = np.union1d(cv_edges, cc_edges)
c_unique_vetices = np.unique(cv_vertices)

c_edge_on_vertices = descriptor.ds.edgesOnVertex[c_unique_vetices]

vertex_mask = np.logical_and.reduce(
    np.isin(c_edge_on_vertices, c_edge_union), axis=1
)

# update edge mask
egde_mask = np.isin(c_edge_on_vertices.values[vertex_mask], cc_edges)

edge_idx = c_edge_on_vertices.values[vertex_mask][egde_mask]
cc_cells = descriptor.ds.cellsOnEdge[edge_idx]

nCells = cc_cells.shape[0]

xc_unsorted = np.insert(ds.xCell[cc_cells].values, 2, np.nan, axis=1)
yc_unsorted = np.insert(ds.yCell[cc_cells].values, 2, np.nan, axis=1)

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(
    ax, descriptor, mask, cmap="binary_r", alpha=0.5, ec="k", lw=1
)

scatter = ax.scatter(
    ds.xCell, ds.yCell, c=mask, cmap="binary_r", s=2.5, ec="k", lw=0.1
)

ax.plot(
    xc_unsorted.ravel(),
    yc_unsorted.ravel(),
    c="tab:orange",
    lw=lw,
    label="Cell",
)

legend = ax.legend(loc="lower right")

ax.add_artist(legend)

kw = {"fmt": "{x!s:}", "func": lambda x: ~x.astype(bool)}
legend1 = ax.legend(
    *scatter.legend_elements(**kw), loc="lower left", title="Classes"
)
ax.add_artist(legend1)

ax.set_xlim(-0.25, 0.25)
ax.set_ylim(-0.25, 0.25);

**Figure 7.** Boolean mask plotted on cells and  discretly plotted at cell centers. Contour boundary created by connecting cell centers of the boundary cells is show in orange. 


## Interface

With a basic sketch of the algorithmic approach outlined above, let's now discuss how we will implement this in practice and how we envision a user interacting with it. 
The goal is to provide contouring ability in `mosaic` where the interface to our contouring functions are as close as possible to `matplotlib` interface. Such that users can capitialize on their previous experience with `matplotlib` and seamlessly contour *directly* on the unstructed mesh. 

Ideally, we will implement something like: 
```python 
mosaic.contour(ax, descriptor, field, ...)
mosaic.contourf(ax, descriptor, field, ...)
```
where `...` are the **exact** same keyword positional and key-word arguments as the `matplotlib` comensurate functions. 

**To Do**:
- Explain the `matplotlib.contour.ContourSet` object and how we plan to subclass for seamless integration. 

- Explain `countrpy.CountrGenerator`, the various line and fill type, and how they integrate with `matplotlib` (through Path objects). 

  - https://contourpy.readthedocs.io/en/v1.3.3/user_guide/calculate/fill_type.html#fill-type



## Unfilled Contours

We'll begin with unfilled contours (i.e. `mosaic.contour`), as they are simpler task. 

Instead of such a simple gaussian field as we consider before, let's work with something more complicated:
$$
f(x, y) = A \sin \left(\frac{N \pi x}{L_{\rm x}}\right) \cos \left(\frac{N \pi y}{L_{\rm y}}\right)
$$
where:
  - $A$ is the amplitude
  - $N$ is the period
  - $L_{\rm x}$ and $L_{\rm y}$ as the domain lengths in the $x$ and $y$ directions, respectively

We'll also slightly increase the size of the mesh to be $50 \times 50$ cells.

In [ ]:
ds = planar_hex_mesh(100)

field = sinusoid(ds.xCell, ds.yCell)
descriptor = mosaic.Descriptor(ds)

In [ ]:
def regular_coord(ds, N=100):
    return np.linspace(float(ds.min()), float(ds.max()), N)


N = int(ds.sizes["nCells"] ** 0.5)
reg_x = regular_coord(ds.xCell, N=N)
reg_y = regular_coord(ds.yCell, N=N)

rect_grid = np.meshgrid(reg_x, reg_y)
rect_data = sinusoid(*rect_grid)

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(ax, descriptor, field, vmax=2.0, vmin=-2.0, aa=False)
fig.colorbar(pc, shrink=0.5, extend="both");

**Figure 6.** Checkerboard field, evaluated on a $100 \times 100$ cell mesh.


A first step, let's create the boolean 

In [ ]:
mask = field < 0.0

In [ ]:
fig, ax = double_pannel()

pc = mosaic.polypcolor(
    ax[0], descriptor, mask, cmap="binary", alpha=1 / 3, ec="k", lw=1 / 5
)

ax[1].contour(*rect_grid, rect_data, levels=[0.0], colors="tab:blue")
ax[1].contourf(
    *rect_grid, rect_data, levels=[-2.0, 0.0], colors="k", alpha=1 / 3
);

**Figure 7.** (Left) Boolean mask of $f(x, y) < 0.0$ evaluated on the MPAS mesh. (Right) The same checkerboard field, evaluated and contoured on a regular quadrilateral mesh. Blue lines are the contour lines and grey is the filled contour area. 

**Boundaries**

We can see from the `matplotlib.pyplot.contour` example above (RHS of Figure 7), that contour lines should be discontinuous when they intersect with a boundary. For example, while the plotted mask flows plot boundary, the contour lines abruptly end. They do not from closed loops by using the boundary. We'll mimic this behavior in `mosaic`. 

In [ ]:
fig, ax = double_pannel()

kwargs = {
    "levels": [-1.0, 0.0, 1.0],
    "colors": ["tab:blue", "tab:orange", "tab:red"],
    "linestyles": "-",
}

mosaic.polypcolor(ax[0], descriptor, field, alpha=1 / 3, ec="k", lw=1 / 5)
mosaic.contour(
    ax[0],
    descriptor,
    field,
    alpha=1.0,
    **kwargs,
)

ax[1].pcolormesh(*rect_grid, rect_data, alpha=1 / 3, lw=1 / 5)
ax[1].contour(*rect_grid, rect_data, **kwargs)

ax[0].set_title(r"\texttt{mosiac.contour}")
ax[1].set_title(r"\texttt{plt.contour}");

**Figure 8**. (Left) `mosaic.contour` result for contours at $-1, 0, 1$ (blue, orange, red) of the checkerboard field, evaluated on the MPAS mesh. (Right) `plt.contour` of the same checkerboard field on a regular quadrilateral grid for comparison. 

As we can see above, we get good agreement between our `mosaic.contour` implementation and the `plt.contour` result. The major difference is the jagged appearance of the MPAS contours due to the coarse resolution of the mesh, **but** this is desired feature of implementation. `plt.contour` does linear interpolation, which produces the smooth boundaries, but that is not something we want (or at least on all the time). Future work will investigate adding optional support for smooth contours. 

## Filled Contours

Filled contours, specifically the possibility of interior boundaries, presents a challenge to our treatment so far mesh contours as graphs.
To illustrate this challenge we will visualize two, multi-component, [isomorphic](https://en.wikipedia.org/wiki/Graph_isomorphism) graphs, which according to graph theory are equivalent graphs. 

In [ ]:
# --------- Build a graph: two disconnected 8-cycles ---------
n = 8
G = nx.disjoint_union(
    nx.cycle_graph(n), nx.cycle_graph(n)
)  # nodes 0..7 and 8..15


# Helper: place nodes evenly on a circle
def circle_pos(nodes, center=(0.0, 0.0), radius=1.0, phase=0.0):
    cx, cy = center
    pos = {}
    for i, node in enumerate(nodes):
        theta = phase + 2.0 * np.pi * i / len(nodes)
        pos[node] = (cx + radius * np.cos(theta), cy + radius * np.sin(theta))
    return pos


outer_nodes = list(range(8))
inner_nodes = list(range(8, 16))

# --------- Embedding 1: Nested rings ---------
pos_nested = {}
pos_nested.update(
    circle_pos(outer_nodes, center=(+1, 0), radius=1.5, phase=0.0)
)
pos_nested.update(
    circle_pos(inner_nodes, center=(+1, 0), radius=0.75, phase=0.2)
)

# --------- Embedding 2: Separate rings (no containment) ---------
pos_separate = {}
pos_separate.update(
    circle_pos(outer_nodes, center=(0, 0), radius=1.5, phase=0.0)
)
pos_separate.update(
    circle_pos(inner_nodes, center=(+3, 0), radius=0.75, phase=0.2)
)


# --------- Visualization: same graph, different coordinates ---------
def draw_embedding(ax, pos, title):
    # Color by component membership
    node_colors = []
    for v in G.nodes():
        node_colors.append("tab:blue" if v in outer_nodes else "tab:orange")

    nx.draw(
        G,
        pos=pos,
        ax=ax,
        with_labels=True,
        node_color=node_colors,
        edge_color="gray",
        node_size=150,
        font_size=8,
        width=2,
    )
    ax.set_title(title)
    ax.set_aspect("equal")
    # ax.axis("off")


fig, ax = double_pannel()
draw_embedding(ax[0], pos_nested, "nested geometry")
draw_embedding(ax[1], pos_separate, "separated geometry")
plt.show()

**Figure 9**. Two multi-componnet isomorphic graphs, plotted using their coordinate information.

As far a graph theory is concerned, the two graphs above are equivalent.
Because the coordinate information to plot these graphs, we can see an important distinction: the graph on the left is nested where the graph on the right is separated. 

For filled contours, **Figure 9.** is a crude, but illustrative, example of a contour with an interior boundary.
So far we only used the mesh connectivity information to create the unfilled contour boundaries, where coordinate information has only been used to plotting.
**But**, for filled contours after creating the contour boundaries from the connectivity information, we'll need to use coordinate info to recursive search of interior boundaries within the resulting contours.

We can efficiently recursively search for interior boundaries using [`shapely.STRtree`](https://shapely.readthedocs.io/en/stable/strtree.html#shapely.STRtree), which uses the bounding boxes of each geometry to query for containment. 


### Relative Ordering

One final `matplotlib` consideration, is the relative ordering (ie., clockwise vs counter-clockwise) of interior boundaries relative to their exterior boundaries. 

In order for an interior boundary to be unfilled, as intended, it's relative order must be opposite the relative order of the exterior boundary.
For more information about this, refer to the `matplotlib` documentation: [here](https://matplotlib.org/stable/gallery/shapes_and_collections/donut.html#sphx-glr-gallery-shapes-and-collections-donut-py). 

Following the `matplotlib` documentation

In [ ]:
def wise(v):
    if v == 1:
        return "CCW"
    return "CW"


def make_circle(r):
    t = np.arange(0, np.pi * 2.0, 0.01)
    t = t.reshape((len(t), 1))
    x = r * np.cos(t)
    y = r * np.sin(t)
    return np.hstack((x, y))


Path = mpath.Path

fig, ax = single_pannel((5.5, 5.5))
# fig, ax = plt.subplots(layout='constrained')

inside_vertices = make_circle(0.5)
outside_vertices = make_circle(1.0)
codes = (
    np.ones(len(inside_vertices), dtype=mpath.Path.code_type)
    * mpath.Path.LINETO
)
codes[0] = mpath.Path.MOVETO

for i, (inside, outside) in enumerate(((1, 1), (1, -1), (-1, 1), (-1, -1))):
    # Concatenate the inside and outside subpaths together, changing their
    # order as needed
    vertices = np.concatenate(
        (outside_vertices[::outside], inside_vertices[::inside])
    )
    # Shift the path
    vertices[:, 0] += i * 2.5
    # The codes will be all "LINETO" commands, except for "MOVETO"s at the
    # beginning of each subpath
    all_codes = np.concatenate((codes, codes))
    # Create the Path object
    path = mpath.Path(vertices, all_codes)
    # Add plot it
    patch = mpatches.PathPatch(path, facecolor="#885500", edgecolor="black")
    ax.add_patch(patch)

    ax.annotate(
        f"Outside {wise(outside)},\nInside {wise(inside)}",
        (i * 2.5, -1.5),
        va="top",
        ha="center",
    )

ax.set_xlim(-2, 10)
ax.set_ylim(-3, 2);

**Figure 10**. Illustration of the importance of the interior vs. exterior relative ordering for rendering interior boundaries. 


So, with above considerations in mind let's return to the same sine field, evaluated on a $100 \times 100$ cell mesh.  

In [ ]:
fig, ax = single_pannel()

pc = mosaic.polypcolor(ax, descriptor, field, vmax=2.0, vmin=-2.0, aa=False)

fig.colorbar(pc, shrink=0.5, extend="both");

**Figure 11.** Checkerboard field, evaluated on a $100 \times 100$ cell mesh.

Like filled contouring, our first step will be to create the boolean mask.
Because this is a filled contour we will need both an upper and lower bound (cf., unfilled). 

In [ ]:
cell_mask = (field > -1.0) & (field < 0.0)

In [ ]:
fig, ax = double_pannel()

mosaic.contour(
    ax[0],
    descriptor,
    field,
    levels=[-1.0, 0.0],
    colors="tab:blue",
    linestyles="-",
)
mosaic.polypcolor(
    ax[0],
    descriptor,
    cell_mask,
    cmap="binary",
    alpha=1 / 3,  # ec='k', lw=1/5
)

ax[1].contour(
    *rect_grid,
    rect_data,
    levels=[-1.0, 0.0],
    colors="tab:blue",
    linestyles="-",
)
ax[1].contourf(
    *rect_grid, rect_data, levels=[-1.0, 0.0], colors="k", alpha=1 / 3
);
# fig.savefig("figs/test.pdf", bbox_inches='tight')

**Figure 12.** (Left) Boolean mask of $-1.0 < f(x, y) < 0.0$ evaluated on the MPAS mesh. (Right) The same checkerboard field, evaluated and contoured on a regular quadrilateral mesh. The grey is the filled contour area. 

As a first step, we will use the exact same steps as unfilled contours to extract and sort the contour boundaries using the connectivity information.
One important difference between the filled (grey) and unfilled (blue lines) is that the filled contours follow the mesh boundary.

In [ ]:
mpas_contour_generator = MPASContourGenerator(descriptor, field)
graph = mpas_contour_generator._create_contour_graph(cell_mask, filled=True)
polys = mpas_contour_generator._split_and_order_graph(graph)

In [ ]:
fig, ax = single_pannel()

for p in polys:
    ax.scatter(*p.T, s=1)

**Figure 13.** The contour boundaries identified just using connectivity information.
Each unique contour component has it's own color.
Notice the pink and brown boundaries are really interior boundaries of the blue contour, but just using connectivity information alone is not enough to identify this. 

We will now used the coordinate information to recursively search for containment of our contour components.
If we find an interior boundary during this recursive search, we will also ensure it's relative order is opposite it's parent to ensure proper plotting in `matplotlib` once we will the contours. 

In [ ]:
polys = mpas_contour_generator._split_and_order_graph(graph)
codes = mpas_contour_generator._assemble_contour_codes(polys)

polys, codes = mpas_contour_generator._sort_filled_contours(polys, codes)

In [ ]:
fig, ax = single_pannel()

for _poly in polys:
    ax.scatter(*_poly.T, s=1.0)

**Figure 14.** The contour boundaries identified after recursively search for containment using the coordinate information.
Notice the interior boundaries of the blue contour are now blue as we'd expect. 

With all this in hand, we now have a fully functional filled contour algorthinm. 
Now let's see it in practice and compare to our `matplotlib` regularly spaced reference. 

In [ ]:
fig, ax = double_pannel()

kwargs = {
    "levels": [-1.0, 0.0, 1.0],
    "colors": ["tab:blue", "tab:orange"],
    "linestyles": "-",
}

mosaic.polypcolor(ax[0], descriptor, field, alpha=1 / 3, ec="k", lw=1 / 5)
mosaic.contourf(
    ax[0],
    descriptor,
    field,
    **kwargs,
)

ax[1].pcolormesh(*rect_grid, rect_data, alpha=1 / 3, lw=1 / 5)
ax[1].contourf(*rect_grid, rect_data, **kwargs)

ax[0].set_title(r"\texttt{mosiac.contour}")
ax[1].set_title(r"\texttt{plt.contour}");

**Figure 15.** `mosaic.contourf` result for filled contour levels of $-1, 0, 1$ of the checkerboard field, evaluated on the MPAS mesh. (Right) `plt.contourf` of the same checkerboard field on a regular quadrilateral grid for comparison. 

As we can see above, we also get good agreement between our `mosaic.contourf` implementation and the `plt.contourf` result.
Again, the major difference is the jagged appearance of the MPAS contours due to the coarse resolution of the mesh, **but** this is desired feature of implementation. 

## Future Work

### Smooth (interpolated) contours:
  - https://www.boristhebrave.com/2018/04/15/marching-cubes-tutorial/
  - https://www.cs.wustl.edu/~taoju/cse554/lectures/lect04_Contouring_I.pdf
  - https://www.cse.wustl.edu/~taoju/research/dualContour.pdf
  - https://en.wikipedia.org/wiki/Marching_squares

### Marching Squares implementation: 
  - `skimage` has a `cython` based marching squares implementation that we could probably easily port to marching triangles.
  - https://github.com/scikit-image/scikit-image/blob/main/src/skimage/measure/_find_contours_cy.pyx